In [1]:
import os

In [6]:
# напишем requirements.txt
requirements = """
scikit-learn==1.7.2
joblib
fastapi
uvicorn
numpy
prometheus_client
python-multipart
"""

with open("requirements.txt", "w", encoding="utf-8") as f:
    f.write(requirements)

print("✅ requirements.txt создан:")

✅ requirements.txt создан:


In [25]:
# напишем наш мини API с эндпоинтами
main = """

from fastapi import FastAPI, Response, Form
from fastapi.responses import HTMLResponse
import numpy as np
import os
import joblib
import logging
from prometheus_client import Counter, Histogram, generate_latest, CONTENT_TYPE_LATEST
import time

# Настройка логов
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# Счётчик запросов
requests_total = Counter("requests_total", "Total number of requests", ["endpoint"])

# Гистограмма для латентности (время выполнения запроса)
request_latency = Histogram(
    "request_latency_seconds",
    "Request latency in seconds",
    ["endpoint"]
)

app = FastAPI()
VERSION = os.getenv("MODEL_VERSION", "v1.0.0")
# Загружаем модель (предварительно сохрани её через joblib.dump)
MODEL_PATH = os.getenv("MODEL_PATH", "models/model.pkl")
model = joblib.load(MODEL_PATH)

@app.get("/health")
def health():
    start = time.time()
    requests_total.labels(endpoint="/health").inc()
    logger.info("Зашли в Health")
    response = {"status": "ok", "version": VERSION}
    request_latency.labels(endpoint="/health").observe(time.time() - start)
    return response

@app.get("/ui", response_class=HTMLResponse)
def ui_form():
    start = time.time()
    requests_total.labels(endpoint="/ui").inc()
    html_content = '''
    <html>
        <head>
            <title>ML Wine Prediction UI</title>
        </head>
        <body>
            <h2>Введите значения признаков для предсказания</h2>
            <form action="/predict" method="post">
                <label for="request">Фичи (через запятую):</label><br>
                <input type="text" id="x" name="request" value="1.0,2.0,3.0"><br><br>
                <input type="submit" value="Отправить">
            </form>
        </body>
    </html>
    '''
    request_latency.labels(endpoint="/ui").observe(time.time() - start)
    return HTMLResponse(content=html_content)

@app.post("/predict")
def predict(request: str = Form(...)):
    start = time.time()
    requests_total.labels(endpoint="/predict").inc()
    features = [float(val.strip()) for val in request.split(',')]
    logger.info(f"Начинаю предикт для интпута: {features}")
    X = np.array(features).reshape(1, -1)
    y_pred = model.predict(X)[0]
    logger.info(f"Получен предикт: {y_pred}")
    response = {
        "status": "ok",
        "prediction": int(y_pred),
        "version": VERSION
    }
    request_latency.labels(endpoint="/predict").observe(time.time() - start)
    return response

@app.get("/metrics")
def metrics():
    return Response(generate_latest(), media_type=CONTENT_TYPE_LATEST)

@app.get("/sleep")
def health():
    start = time.time()
    requests_total.labels(endpoint="/sleep").inc()
    logger.info("Зашли в Sleep")
    time.sleep(2)
    response = {"status": "ok", "sleep": "ok"}
    request_latency.labels(endpoint="/sleep").observe(time.time() - start)
    return response

"""

with open("app/main.py", "w", encoding="utf-8") as f:
    f.write(main)

print("✅ main.py создан:")

✅ main.py создан:


In [14]:
# создадим докер для запуска API с моделью
dockerfile_content = """
# Базовый образ
FROM python:3.10-slim

# Переменные окружения
ENV MODEL_PATH=/app/models/model.pkl \\
    MODEL_VERSION=v1.0.0

# Рабочая директория
WORKDIR /app

# Копирование зависимостей и установка
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Копируем весь проект
COPY . .

# Точка входа
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8080"]
"""

dockerignore = """
__pycache__/
*.pyc
.git
.venv
tests/
"""

with open("Dockerfile", "w") as f:
    f.write(dockerfile_content.strip())

print("✅ Dockerfile создан:")

with open(".dockerignore", "w") as f:
    f.write(dockerignore.strip())

print("✅ .dockerignore создан:")

✅ Dockerfile создан:
✅ .dockerignore создан:


In [23]:
# создадим конфиг прометеуса, чтобы он мониторил метрики с api (в таргете ставим имя работы и порт так как мл сервис будет в докер композе)
prometheus_content = """
scrape_configs:
  - job_name: "ml_service"
    static_configs:
      - targets: ["ml_service:8080"]
"""

with open("prometheus.yml", "w") as f:
    f.write(prometheus_content.strip())

print("✅ prometheus.yml создан:")


✅ prometheus.yml создан:


In [15]:
# Напишем докер композе для запуска 3 сервисов одновременно
docker_compose_content = """
version: "3.9"
services:
    prometheus:
      image: prom/prometheus:latest
      volumes:
        - ./prometheus.yml:/etc/prometheus/prometheus.yml
      ports:
        - "9090:9090"

    grafana:
      image: grafana/grafana:latest
      ports:
        - "3000:3000"
      depends_on:
        - prometheus

    ml_service:
      image: ml-service-dz4-ml-in-prodaction:v1
      ports:
        - "8000:8080"
"""

with open("docker-compose.yml", "w") as f:
    f.write(docker_compose_content.strip())

print("✅ docker-compose.yml создан:")

✅ docker-compose.yml создан:
